In [1]:
import insightface
import urllib
import urllib.request
import cv2
import json
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
face_embeddings = []
cosine_threshold = 0.5
color = (0, 0, 255)
font = cv2.FONT_HERSHEY_SIMPLEX
ctx_id = 0

In [3]:
cap = cv2.VideoCapture('videos/video1.mp4')

In [4]:
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
size = (frame_width, frame_height)

result = cv2.VideoWriter('result.avi', cv2.VideoWriter_fourcc(*'MJPG'), 10, size)

In [5]:
model = insightface.app.FaceAnalysis()
model.prepare(ctx_id = ctx_id, nms=0.4)

[32, 16, 8] {'32': {'SCALES': (32, 16), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '16': {'SCALES': (8, 4), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}, '8': {'SCALES': (2, 1), 'BASE_SIZE': 16, 'RATIOS': (1.0,), 'ALLOWED_BORDER': 9999}}
use_landmarks True


In [6]:
def SaveFace(frame, face, faceNo):
    # Write face
    bbox = face.bbox.astype(np.int).flatten()
    cropped_face = frame[bbox[1] - 20:bbox[3] + 20, bbox[0] - 20:bbox[2] + 20]
    status = cv2.imwrite('detectedfaces/FaceNo' + str(faceNo) + '.png', cropped_face)
    # print('Face No.' + str(faceNo) + ' is written to file-system :', status)
    
    # Add face data to json dataset
    data = {'faceNo': str(faceNo), 
            'embeddingVal': str(face.embedding),
            'bbox': str(bbox),
            'landmark': str(face.landmark.astype(np.int).flatten())
           }
    
    return data

In [7]:
# Write json file
def write_json(data, filename='data.json'): 
    with open(filename,'w') as f: 
        json.dump(data, f, indent=4)

In [ ]:
# Extract face and it's information in the input video
faceNo = 0
frameNo = 0
jsonData = []

while(True):
    ret, frame = cap.read()
    nimg = frame
    
    faces = model.get(nimg)
    for idx, face in enumerate(faces):
        bbox = face.bbox.astype(np.int).flatten()
        face_json_data = SaveFace(nimg, face, faceNo)
        jsonData.append(face_json_data)
        write_json(jsonData)
        
        faceNo += 1
        
    print('Frame No:' + str(frameNo) + '. Number of detected face: ' + str(faceNo))
    frameNo += 1

In [ ]:
# Save video with face detection and tagged id per face
while(True):
    frameNo += 1
    ret, frame = cap.read()
    nimg = frame
    
    faces = model.get(nimg)
    for idx, face in enumerate(faces):
        bbox = face.bbox.astype(np.int).flatten()
        cosine_dist, face_id = MaxCosineSimilarity(face.embedding, face_embeddings)
        
        cv2.rectangle(nimg, (bbox[0] - 20, bbox[1] - 20), (bbox[2] + 20, bbox[3] + 20), color, 2)
        
        if cosine_dist < cosine_threshold:
            face_embeddings.append(face.embedding)
            faceNo += 1
            cv2.putText(nimg, str(faceNo), (bbox[0] - 5, bbox[1] - 5), font, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
        else:
            cv2.putText(nimg, str(face_id), (bbox[0] - 5, bbox[1] - 5), font, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
            
    print('frame No:' + str(frameNo) + '. Number of detected face: ' + str(faceNo))
    cv2.putText(nimg, 'Total face detected: ' + str(faceNo), (30, 30), font, 1.2, (0, 0, 255), 2, cv2.LINE_AA)
    
    # cv2.imwrite('frames/frameNo' + str(frameNo) + '.png', nimg)
    result.write(nimg)
    clear_output(wait=True)
cap.release()
result.release()